In [1]:
from sklearn.metrics import confusion_matrix, recall_score, f1_score

import sys
sys.path.append(r"D:\work\federated-boosted-dp-trees")
sys.path.append(r"D:\work\phase_tree")

from util import *

from federated_gbdt.models.gbdt.private_gbdt import PrivateGBDT
from experiments.experiment_helpers.data_loader import DataLoader
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_svmlight_file
from pathlib import Path
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [2]:
dname = 'cod-rna'

def get_dp_tr(dname):
    ds = auto_read_dataset(dname)
    dp_tr_model = PrivateGBDT(num_trees=100, epsilon=0.5, split_method="totally_random", 
                            sketch_type="uniform", dp_method="gaussian_cdp")
    dp_tr_model = dp_tr_model.fit(ds[0], ds[1])
    return dp_tr_model

ds = auto_read_dataset(dname)

clf = cgb(verbose=False)
clf.fit(ds[0], ds[1])
clf.score(ds[2], ds[3])

# clf = get_dp_tr(dname)

0.9636348366507097

In [3]:
# shadow dataset

x1 = [x for x in ds[0]]
x2 = [x for x in ds[2]]
xs = x1 + x2
nf = len(xs[0])
for x in xs:
    for i in range(nf):
        x[i] += np.random.laplace(0, 100)

nn = len(xs) >> 1

# d4t
# ys = [clf.predict(x) for x in xs]

# dp-tr
ys = clf.predict(np.array(xs))

xt = xs[nn:]
yt = xs[nn:]

xs = xs[:nn]
ys = ys[:nn]

In [4]:
# shadow model & adv dataset

nm = 16
n = len(xs)
j = 0
l = n // nm

xa = []
ya = []

sms = [cgb(verbose=False) for _ in range(nm)]
for i in range(nm):
    sms[i].fit(xs[j:j+l], ys[j:j+l])
    for x in xs[j:j+l]:
        xa.append(sms[i].predict_proba(x))
        ya.append(1)
    j += l

for x in xt:
    md = random.choice(sms)
    xa.append(md.predict_proba(x))
    ya.append(0)

In [5]:
# adv model
amd = cgb(verbose=False)
amd.fit(xa, ya)

In [6]:
ds = auto_read_dataset(dname)

xxs = [x for x in ds[0]]
xxs = xxs[:len(ds[2])]
yys = [1] * len(xxs)

xxs += [x for x in ds[2]]
yys += [0] * len(ds[2])

dd = ds[0][:500]
for x in dd:
    for i in range(nf):
        x[i] += np.random.laplace(0, 100)

xxs += dd.tolist()
yys += [0] * len(dd)

In [7]:
def infer(md, tar, x):
    p = tar.predict(x)
    return md.predict(p)

In [8]:
# pred = [infer(amd, clf, x) for x in np.array(xxs)]

pred = amd.predict(clf.predict_proba(np.array(xxs)))

In [9]:
# acc tp/fp auc f1 recall

mt = confusion_matrix(yys, pred)
tp = mt[1][1]
fp = mt[0][1]
r = tp / fp

accuracy_score(yys, pred) *100, r, roc_auc_score(yys, pred), f1_score(yys, pred), recall_score(yys, pred)

(49.64629431603191,
 0.9523674510915793,
 0.4978279585976726,
 0.523229097706297,
 0.5642059292852943)